In [1]:
# importar librerias
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from urllib.parse import urljoin
from openai import OpenAI
import pandas as pd
from readability import Document


### Estructura
- Scraping de todos los links de la web (guardamos link, titulo de cada una y un id)
- Utilizamos un LLM (gemma 3) para filtrar las noticias mas relevantes (formato Json)
- Las noticias más importantes resumirlas en 30-40 caracteres
- Enviar el titulo, la noticia resumida y el link a un bot de telegram



## Scraping de la web

In [34]:
# url de noticias de mallorca
url = "https://www.ultimahora.es/sucesos.html"
id = 0
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

try:
    response = requests.get(url, headers=headers, timeout=10)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, 'html.parser')
    news_list = []

    # Encontrar todos los contenedores de noticias
    news_containers = soup.find_all('div', class_='news-item')

    for container in news_containers:
        # Extraer titulo
        title_element = container.find(['h2', 'h3'], class_='news-heading')
        if not title_element:
            continue

        link_element = title_element.find('a', href=True)
        if not link_element:
            continue

        title = title_element.get_text(strip=True)
        link = urljoin(url, link_element['href'])

        news_list.append({
            'title': title,
            'url': link,
            'id' : id
        })
        id = id + 1

    # Mostrar resultados
    print(f"Noticias encontradas: {len(news_list)}")
    for idx, news in enumerate(news_list, 1):
        print(f"\nNoticia {idx}:")
        print(f"Título: {news['title']}")
        print(f"Enlace: {news['url']}")
        print(f"id: {news['id']}")

except requests.exceptions.RequestException as e:
    print(f"Error de conexión: {e}")
except Exception as e:
    print(f"Error: {e}")

Noticias encontradas: 32

Noticia 1:
Título: Agreden y roban a un hombre que sufrió una indisposición en el coche tras sacar 500 euros en el banco
Enlace: https://www.ultimahora.es/sucesos/ultimas/2025/05/29/2396345/sucesos-mallorca-joven-carne-ayuda-aparcar-hombre-sintio-indispuesto-son-gotleu-roba-500-euros-tras-agredirle.html
id: 0

Noticia 2:
Título: Condenado un guardia civil por acoso telefónico a un examigo en Marratxí
Enlace: https://www.ultimahora.es/sucesos/ultimas/2025/05/29/2396861/sucesos-mallorca-condenado-guardia-civil-por-acoso-telefonico-examigo-marratxi.html
id: 1

Noticia 3:
Título: Denuncian robos en el cementerio de Llucmajor: «No respetan a los muertos»
Enlace: https://www.ultimahora.es/sucesos/ultimas/2025/05/29/2393421/denuncian-robos-cementerio-llucmajor-respetan-muertos.html
id: 2

Noticia 4:
Título: Muere la ciclista que fue arrollada por una furgoneta en Santa Margalida
Enlace: https://www.ultimahora.es/sucesos/ultimas/2025/05/28/2396523/sucesos-mallorca-mue

## Filtro gemma3

In [35]:
from openai import OpenAI

modelo_filtro_titulos = OpenAI( base_url="http://localhost:11434/v1" , api_key= "ollama")


In [36]:
# Crear lista simplificada con solo title e id para ahorrar tiempo y tokens
datos_simplificados = [
    {"title": item["title"], "id": item["id"]}
    for item in news_list
]


In [37]:
msg = [
    {
        "role": "system",
        "content": """Eres un editor experto en análisis de noticias. Tu tarea es:
        
                    1. Analizar esta lista de titulares de noticias
                    2. Seleccionar los 10 más relevantes según estos criterios:
                    - Impacto social
                    - Gravedad del suceso
                    - Relevancia pública
                    - Novedad del acontecimiento

                    REQUISITOS DE RESPUESTA:
                    - Devuelve ÚNICAMENTE un array JSON válido, sin comentario adicionales ni nada mas.
                    - Usa EXACTAMENTE los mismos títulos recibidos
                    - Devuelve exactamente el mismo id relacionado al título de la noticia
                    - Estructura exacta requerida:
                    "noticias_relevantes": [
                            {
                        "titulo": "Texto exacto del titular original",
                        "categoria": "asesinato|violencia|robo|accidente|corrupción|otros",
                        "id": id exacto de la noticia,
                        "impacto_emocional": 0-10,
                        "prioridad": "alta|media|baja"
                        },
                    ]
                    """
    },
    {
        "role": "user",
        "content": f"LISTA DE TITULARES:\n{datos_simplificados}"
    }
  ]

In [57]:
# Comento par que no se ejecute sin quereer otra vez.
response = modelo_filtro_titulos.chat.completions.create(
    model="gemma3:4b",
    messages=msg,
    response_format={"type": "json_object"}  # forzar al modelo a que devulva un json

)

print(json.loads(response.choices[0].message.content))

KeyboardInterrupt: 

In [49]:
# # guardo repsuesta para no ejecutarlo mas veces.
# respuesta_modelo = {"noticias_relevantes": [
#     {"title": "Nuevo escándalo de Kanye West: La Policía investiga un impago de casi medio millón de euros del rapero en Mallorca", "categoria": "otros", "id": 0, "impacto_emocional": 7, "prioridad": "alta"},
#     {"title": "Condenado en Palma el octogenario que entró sin permiso en casa de sus inquilinas para oler sus bragas", "categoria": "violencia", "id": 2, "impacto_emocional": 9, "prioridad": "alta"},
#     {"title": "Una colisión en la Ma-13 provoca varios kilómetros de retenciones a la altura de Santa Maria", "categoria": "accidente", "id": 11, "impacto_emocional": 4, "prioridad": "media"},
#     {"title": "Piden 28 años cárcel a un hombre por abusos sexuales y vejaciones a tres menores de edad en Palma", "categoria": "otros", "id": 12, "impacto_emocional": 10, "prioridad": "alta"},
#     {"title": "Avícola Son Perot abre sus puertas para mostrar la realidad de la granja ante «las imágenes manipuladas»", "categoria": "otros", "id": 18, "impacto_emocional": 8, "prioridad": "alta"},
#     {"title": "Una empleada del hogar roba 6.400 euros y cubertería de plata a la octogenaria a la que cuidaba en Palma", "categoria": "violencia", "id": 33, "impacto_emocional": 7, "prioridad": "alta"},
#     {"title": "Un incendio en una casa de Palma destapa una plantación de marihuana", "categoria": "otros", "id": 32, "impacto_emocional": 6, "prioridad": "media"},
#     {"title": "Presunto caso de violencia machista en un pueblo de Cáceres", "categoria": "violencia", "id": 22, "impacto_emocional": 9, "prioridad": "alta"},
#     {"title": "Una pelea en un hostal de Favara acaba con un fallecido y tres heridos, uno de ellos grave", "categoria": "violencia", "id": 23, "impacto_emocional": 10, "prioridad": "alta"},
#     {"title": "Aparecen pintadas e intentan pegar fuego a los Juzgados de Ibiza", "categoria": "otros", "id": 34, "impacto_emocional": 5, "prioridad": "media"}
# ]}

respuesta_modelo = json.loads(response.choices[0].message.content)


In [50]:
link_por_id = {noticia['id']: noticia['url'] for noticia in news_list}


# Paso 2: añadir el link a cada entrada en respuesta_modelo
for entrada in respuesta_modelo['noticias_relevantes']:
    noticia_id = entrada['id']
    if noticia_id in link_por_id:
        entrada['url'] = link_por_id[noticia_id]
    else:
        entrada['url'] = None  # o lo que prefieras si no encuentra el link



## Resumir urls seleccionadas

In [51]:
def obtener_noticia(url):
    """
    Dada una URL, realiza scraping del contenido principal de la noticia
    y devuelve el texto limpio para ser resumido posteriormente.
    """
    if url:
        try:
            headers = {
                "User-Agent": "Mozilla/5.0"
            }
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()

            # Extraer HTML limpio con Readability
            doc = Document(response.text)
            html_limpio = doc.summary()

            # Parsear con BeautifulSoup para limpiar etiquetas
            soup = BeautifulSoup(html_limpio, 'html.parser')
            texto = soup.get_text(separator='\n')

            # Limpiar espacios en blanco
            texto = '\n'.join([line.strip() for line in texto.splitlines() if line.strip()])

            return texto

        except Exception as e:
            print("[ERROR] ", e)
            return ""
    else:
        return None


def resumir_web(url , modelo):
    texto = obtener_noticia(url)
    if texto:
        # Creamos el mensaje para el modelo
        msg = [
        {
            "role": "system",
            "content": """Eres un editor experto en análisis de noticias. Tu tarea es resumir la 
                        noticia al maximo, de unos aproximadamente 70 caracteres, captando la 
                        atencion del lector. No debes inventar nada, ni ser sensacionalista,
                        pero si debes intentar captar la atencion del lector con el resumen
                        maximo posible.
                        """
        },
        {
            "role": "user",
            "content": f"LISTA DE TITULARES:\n{texto}"
        }
        ]

        # Enviamos el mensaje al modelo junto al texto para resumirlo
        response = modelo.chat.completions.create(
        model="gemma3:4b",
        messages=msg
        )
        # print(response.choices[0].message.content)

        return response.choices[0].message.content
    else:
        return None




In [54]:
modelo_resumen_noticia = OpenAI( base_url="http://localhost:11434/v1" , api_key= "ollama")

for noticias in respuesta_modelo['noticias_relevantes']:
    noticias['resumen_breve'] = resumir_web(noticias['url'] , modelo_resumen_noticia)
    print(noticias['titulo'])
    print(noticias['resumen_breve'])
    print(noticias['url'])
    print("- - - - - - - - - - - - - - - - - - - - - - - - - - - ")


Agreden y roban a un hombre que sufrió una indisposición en el coche tras sacar 500 euros en el banco
Robo y violencia en Palma: Detenido por robar 500€ tras ayudar a víctima. ¡Alarma en Son Gotleu!
https://www.ultimahora.es/sucesos/ultimas/2025/05/29/2396345/sucesos-mallorca-joven-carne-ayuda-aparcar-hombre-sintio-indispuesto-son-gotleu-roba-500-euros-tras-agredirle.html
- - - - - - - - - - - - - - - - - - - - - - - - - - - 
Muere la ciclista que fue arrollada por una furgoneta en Santa Margalida
Ciclista italiana de 49 años fallece tras accidente en Mallorca. Grave lesión tras atropello.
https://www.ultimahora.es/sucesos/ultimas/2025/05/28/2396523/sucesos-mallorca-muere-ciclista-fue-arrollada-por-furgoneta-santa-margalida.html
- - - - - - - - - - - - - - - - - - - - - - - - - - - 
Un hombre, en estado crítico tras ser apuñalado por su novia
**Resumen:**

Mujer apuñala a su novio en Latina; víctima en estado crítico. Investigación en curso.
https://www.ultimahora.es/sucesos/ultimas/20

In [56]:
respuesta_modelo['noticias_relevantes']

[{'titulo': 'Agreden y roban a un hombre que sufrió una indisposición en el coche tras sacar 500 euros en el banco',
  'categoria': 'violencia|robo',
  'id': 0,
  'impacto_emocional': 8,
  'prioridad': 'alta',
  'url': 'https://www.ultimahora.es/sucesos/ultimas/2025/05/29/2396345/sucesos-mallorca-joven-carne-ayuda-aparcar-hombre-sintio-indispuesto-son-gotleu-roba-500-euros-tras-agredirle.html',
  'resumen_breve': 'Robo y violencia en Palma: Detenido por robar 500€ tras ayudar a víctima. ¡Alarma en Son Gotleu!'},
 {'titulo': 'Muere la ciclista que fue arrollada por una furgoneta en Santa Margalida',
  'categoria': 'accidente|muerte',
  'id': 3,
  'impacto_emocional': 9,
  'prioridad': 'alta',
  'url': 'https://www.ultimahora.es/sucesos/ultimas/2025/05/28/2396523/sucesos-mallorca-muere-ciclista-fue-arrollada-por-furgoneta-santa-margalida.html',
  'resumen_breve': 'Ciclista italiana de 49 años fallece tras accidente en Mallorca. Grave lesión tras atropello.'},
 {'titulo': 'Un hombre, en e

## Difusion

In [ ]:
def enviar_telegram(mensaje, bot_token, chat_ids):
    """
    Envía un mensaje a múltiples chat_ids a través de un bot de Telegram.
    
    Parámetros:
        mensaje (str): El contenido del mensaje a enviar.
        bot_token (str): El token del bot de Telegram.
        chat_ids (list): Lista de chat_id (puede ser grupo o usuario).
    """
    for chat_id in chat_ids:
        url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
        payload = {
            "chat_id": chat_id,
            "text": mensaje,
            "parse_mode": "HTML"  # Puedes usar "Markdown" si prefieres
        }
        try:
            response = requests.post(url, data=payload)
            if not response.ok:
                print(f"Error al enviar a {chat_id}: {response.text}")
        except Exception as e:
            print(f"Excepción al enviar a {chat_id}: {str(e)}")


In [ ]:
# from config_usuario import config_telegram

usuario_actual = "abde"  # Puedes pedirlo como parámetro o argumento de función
chat_ids = config_telegram["usuarios"].get(usuario_actual, [])
bot_token = config_telegram["bot_token"]

# Construye el mensaje con tus noticias
for noticia in noticias:
    mensaje = f"<b>{noticia['titulo']}</b>\n{noticia['resumen_breve']}\n{noticia['url']}"
    enviar_telegram(mensaje, bot_token, chat_ids)
